In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\HAL 9004\\.conda\\envs\\spark-polito\\lib\\site-packages\\pyspark'

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Excercise 35") \
    .getOrCreate()

In [3]:
spark

In [4]:
#Exercise 35
#
# Dates associated with the maximum value
# Input: a collection of (structured) textual csv files
#containing the daily value of PM10 for a set of
#sensors
#▪ Each line of the files has the following format: sensorId,date,PM10 value (μg/m3)\n
#Output: the date(s) associated with the maximum value of PM10
#▪ Store the result in an HDFS folder

## With RDD's

In [5]:
sc = spark.sparkContext

In [6]:
sc

<SparkContext master=local[*] appName=Excercise 35>

In [7]:
rdd_35 = sc.textFile("input_file_35.txt")

In [8]:
rdd_35.take(30)

['s1,2016-01-01,20.5',
 's2,2016-01-01,30.1',
 's1,2016-01-02,60.2',
 's2,2016-01-02,20.4',
 's1,2016-01-03,60.2',
 's2,2016-01-03,52.5']

In [9]:
rdd_35 = rdd_35.map(lambda a: a.split(","))

In [10]:
rdd_35.take(30)

[['s1', '2016-01-01', '20.5'],
 ['s2', '2016-01-01', '30.1'],
 ['s1', '2016-01-02', '60.2'],
 ['s2', '2016-01-02', '20.4'],
 ['s1', '2016-01-03', '60.2'],
 ['s2', '2016-01-03', '52.5']]

In [11]:
rdd_35 = rdd_35.map(lambda a: (a[2],(a[0],a[1])) ) 
rdd_35.take(10)

[('20.5', ('s1', '2016-01-01')),
 ('30.1', ('s2', '2016-01-01')),
 ('60.2', ('s1', '2016-01-02')),
 ('20.4', ('s2', '2016-01-02')),
 ('60.2', ('s1', '2016-01-03')),
 ('52.5', ('s2', '2016-01-03'))]

In [12]:
rdd_35 = rdd_35.sortByKey(ascending = False)
rdd_35.take(10)

[('60.2', ('s1', '2016-01-02')),
 ('60.2', ('s1', '2016-01-03')),
 ('52.5', ('s2', '2016-01-03')),
 ('30.1', ('s2', '2016-01-01')),
 ('20.5', ('s1', '2016-01-01')),
 ('20.4', ('s2', '2016-01-02'))]

In [13]:
top_value = rdd_35.first()[0]

In [14]:
rdd_35_top = rdd_35.filter(lambda x : x[0]== top_value)
rdd_35_top.take(10)


[('60.2', ('s1', '2016-01-02')), ('60.2', ('s1', '2016-01-03'))]

In [15]:
rdd_35_top = rdd_35_top.map(lambda x:x[1][1])
rdd_35_top.take(2)

['2016-01-02', '2016-01-03']

## With DataFrames

In [16]:
df35 = rdd_35.toDF()

In [17]:
df35

DataFrame[_1: string, _2: struct<_1:string,_2:string>]

In [18]:
df35.take(10)

[Row(_1='60.2', _2=Row(_1='s1', _2='2016-01-02')),
 Row(_1='60.2', _2=Row(_1='s1', _2='2016-01-03')),
 Row(_1='52.5', _2=Row(_1='s2', _2='2016-01-03')),
 Row(_1='30.1', _2=Row(_1='s2', _2='2016-01-01')),
 Row(_1='20.5', _2=Row(_1='s1', _2='2016-01-01')),
 Row(_1='20.4', _2=Row(_1='s2', _2='2016-01-02'))]

In [19]:
changedTypedf = df35.withColumn("_1", df35["_1"].cast("double"))


In [20]:
changedTypedf.show()

+----+----------------+
|  _1|              _2|
+----+----------------+
|60.2|{s1, 2016-01-02}|
|60.2|{s1, 2016-01-03}|
|52.5|{s2, 2016-01-03}|
|30.1|{s2, 2016-01-01}|
|20.5|{s1, 2016-01-01}|
|20.4|{s2, 2016-01-02}|
+----+----------------+



In [21]:
import org.apache.spark.sql.functions._

df = df.select(col("*"), substring(col("a"), 4, 6).as("c"))





SyntaxError: invalid syntax (927756963.py, line 3)

In [ ]:
df33 = df33.orderBy("_1", ascending = False)

In [ ]:
df33.show()

## With SQL

In [ ]:
df33.createOrReplaceTempView("ex33")


In [ ]:
sqlDF = spark.sql("SELECT * FROM ex33")
sqlDF.show()

In [ ]:
sqlDF = spark.sql("SELECT * FROM ex33 ORDER BY _c2 DESC")
sqlDF.show()